In [ ]:
# Importing the libraries
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import LSTM, GRU
from tensorflow.keras.layers import Dropout
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import load_model
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.callbacks import ModelCheckpoint
import itertools
import random
import os

from math import sqrt

import keras_tuner as kt

from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error

np.random.seed(42)
tf.random.set_seed(42)

In [ ]:
df = pd.read_csv("data/sarima_residuals.csv")
residuals = df.values
plt.plot(residuals)

In [ ]:
train_size = int(len(residuals) * 0.67)
test_size = len(residuals) - train_size
train, test = residuals[:train_size], residuals[train_size:]

In [ ]:
scaler = MinMaxScaler()
train_scaled = scaler.fit_transform(train)
test_scaled = scaler.transform(test)

In [ ]:
def create_dataset(dataset, look_back=1):
    dataX, dataY = [], []
    for i in range(len(dataset)-look_back-1):
        a = dataset[i:(i+look_back), :]  #get look_back sequences
        dataX.append(a)
        dataY.append(dataset[i + look_back, 0]) #get the target after look_back sequences
    return np.array(dataX), np.array(dataY)

In [ ]:
look_back = 30
trainX, trainY = create_dataset(train_scaled, look_back)
testX, testY = create_dataset(test_scaled, look_back)

In [ ]:
print("trainX Shape: ",trainX.shape) #[samples, time steps, features]
print("trainY Shape: ",trainY.shape)

print("testX Shape: ",testX.shape) #[samples, time steps, features]
print("testY Shape: ",testY.shape)

In [ ]:
def build_model(hp):
    model = Sequential()
    model.add(
        LSTM(
            hp.Int("input_unit", min_value=32, max_value=128, step=32),
            return_sequences=True,
            input_shape=(5, 1),
        )
    )
    if hp.Boolean("add_middle_lstm"):
        model.add(
            LSTM(
                hp.Int("middle_units", min_value=32, max_value=128, step=32),
                return_sequences=True,
            )
        )
        # model.add(
        #     Dropout(hp.Float("middle_dropout", min_value=0, max_value=0.5, step=0.1))
        # )
    # for i in range(hp.Int('n_layers', 0, 2)):
    #     model.add(LSTM(hp.Int(f'lstm_{i}_units',min_value=32,max_value=128,step=32),return_sequences=True))
    #     model.add(Dropout(hp.Float('Dropout_rate',min_value=0,max_value=0.5,step=0.1)))
    model.add(LSTM(hp.Int(f"last_units", min_value=32, max_value=128, step=32)))
    # model.add(Dropout(hp.Float("last_dropout", min_value=0, max_value=0.5, step=0.1)))
    model.add(Dense(1))
    model.compile(loss="mean_squared_error", optimizer="adam", metrics=["mse"])
    return model

In [ ]:
# Inverse transform and reshape predictions
def inverse_transform_and_reshape(predictions, scaler, shape):
    predictions_copies = np.repeat(predictions, shape[2], axis=-1)
    return scaler.inverse_transform(np.reshape(predictions_copies, (len(predictions), shape[2])))[:, 0]

# stacked_pred = inverse_transform_and_reshape(stacked_test_predictions, scaler, trainX.shape)

# # Original test label
# original_copies_array = np.repeat(testY, trainX.shape[2], axis=-1)
# original_testY = scaler.inverse_transform(np.reshape(original_copies_array, (len(testY), trainX.shape[2])))[:, 0]

In [ ]:
tuner = kt.RandomSearch(
  build_model,
  objective="mse",
  max_trials=10,
  executions_per_trial=3
)

In [ ]:
print(testX.shape)
print(trainX.shape)

In [ ]:
tuner.search(x=trainX, y=trainY, epochs=20, batch_size=128, validation_data=(testX, testY))

In [ ]:
models = tuner.get_best_models(num_models=2)
best_model = models[0]
best_model.summary()


In [ ]:
best_model.fit(trainX, trainY, epochs=20, verbose=0)

In [ ]:
test_predictions = best_model.predict(testX)

In [ ]:
stacked_pred = inverse_transform_and_reshape(test_predictions, scaler, trainX.shape)

# Original test label
original_copies_array = np.repeat(testY, trainX.shape[2], axis=-1)
original_testY = scaler.inverse_transform(np.reshape(original_copies_array, (len(testY), trainX.shape[2])))[:, 0]

In [ ]:
print("Stacked Model:")
mse = mean_squared_error(original_testY, stacked_pred)
print("Mean Squared Error (MSE):", mse)
rmse = sqrt(mse)
print("Root Mean Squared Error (RMSE):", rmse)

In [ ]:
# Plot for Stacked Model
plt.plot(original_testY[0:100], label='Actual')
plt.plot(stacked_pred[0:100], label='Stacked Model Predicted')
plt.xlabel('Time')  # Use `xlabel` instead of `set_xlabel`
plt.ylabel('Passengers')
plt.legend()
plt.title('Actual vs. Stacked Model Predicted Passengers')  # Use `title` instead of `set_title`
plt.tight_layout() 
plt.show()

In [ ]:
plt.plot(residuals)